In [ ]:
# install the required packages
%pip install transformers torch pandas numpy psycopg2


In [42]:
# import the required packages
import pandas as pd
import psycopg2
from transformers import CLIPModel, CLIPProcessor
import torch
import numpy as np
from psycopg2.extensions import register_adapter, AsIs


In [41]:


file_path = '/Users/janezhang/final--project/pgvector/NLP/text-search/text-embedding-filled.csv'  
df = pd.read_csv(file_path)

empty_cells = df.isnull() | (df == '')


empty_rows_ids = df.loc[empty_cells.any(axis=1), 'id']

print("Rows with empty cells (IDs):")
print(empty_rows_ids.tolist())

Rows with empty cells (IDs):
[]


In [33]:

# Read the CSV file
file_path = '/Users/janezhang/final--project/pgvector/NLP/text-search/text-embedding.csv'  # 替换为你的CSV文件路径
df = pd.read_csv(file_path, dtype={'article_id': str, 'product_code': str})

# Find the positions of all empty cells
empty_cells = df.isnull() | (df == '')

# Get the 'id' column values of rows containing empty cells
empty_rows_ids = df.loc[empty_cells.any(axis=1), 'id']

# Output the IDs of rows with empty cells
print("Rows with empty cells (IDs):")
print(empty_rows_ids.tolist())

# Fill all empty cells with spaces
df.fillna(' ', inplace=True)  
df.replace('', ' ', inplace=True)  

# Save the processed data to a new CSV file
output_file_path = 'text-embedding-filled.csv'
df.to_csv(output_file_path, index=False)
print(f"Processed data saved to '{output_file_path}'")


Rows with empty cells (IDs):
[1468, 2645, 2646, 2743, 2744, 2745, 2746, 2747, 2748, 2749, 2750, 3878, 3879, 3880, 3881, 3882, 3883, 3884, 3885, 3886, 3887, 3888, 5248, 5249, 5250, 5251, 5252, 5253, 5254, 5255, 5256, 5257, 5258, 5259, 5260, 6630, 6631, 6632, 6633, 6634, 6635, 6636, 6637, 7128, 7129, 7130, 8100, 8101, 8102, 8103, 8104, 8105, 8291, 8292, 8293, 8958, 10133, 10134, 10280, 10313, 10314, 10315, 12079, 12080, 13020, 13021, 13022, 13023, 13180, 13249, 13303, 13304, 13517, 13518, 13519, 13520, 13521, 13522, 13687, 14115, 14116, 14117, 14118, 14119, 14120, 14121, 14338, 14339, 14342, 14361, 14362, 14363, 14364, 14365, 14437, 14692, 14693, 14694, 15396, 15397, 15631, 15632, 15633, 15634, 15635, 15636, 15637, 15638, 15639, 15803, 15804, 15805, 15806, 15807, 15808, 15809, 15810, 15811, 15812, 15813, 15814, 15815, 15816, 15817, 15840, 15841, 15842, 15843, 16027, 16044, 16045, 16169, 16170, 16692, 17434, 17435, 17501, 17502, 17503, 17504, 17505, 17544, 17545, 17546, 17547, 17999, 1800

In [43]:
# 1. convert numpy types into PostgreSQL-recognizable types
def add_numpy_adapter():
    
    def adapt_numpy_int64(numpy_int):
        return AsIs(numpy_int)
    
    def adapt_numpy_float64(numpy_float):
        return AsIs(numpy_float)
    
    
    register_adapter(np.int64, adapt_numpy_int64)
    register_adapter(np.float64, adapt_numpy_float64)

In [44]:
# 1. Read the dataset and extract the required columns
def load_and_extract_columns(file_path, columns):
    """
    Read specified columns from a CSV file
    :param file_path: The path to the CSV file
    :param columns: A list of column names to extract
    :return: A DataFrame containing the specified columns

    """
    df = pd.read_csv(file_path, usecols=columns)
    return df

In [45]:
def generate_clip_embedding(model, processor, text):
    
    inputs = processor(text=[text], return_tensors="pt", padding=True, truncation=True)
    with torch.no_grad():
        embedding = model.get_text_features(**inputs).cpu().numpy().flatten()  # 生成单行向量
    return embedding

In [46]:
def store_embedding_in_pgvector(conn, table_name, row, embedding):
    
    cursor = conn.cursor()

    
    query = f"""
    INSERT INTO {table_name} (
        article_id, product_code, product_name, product_type, product_group,
        graphical_appearance, colour_group, perceived_colour, 
        department, index_group, section_name, detail_desc, text_vector
    ) VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s)
    """

    
    values = (
        row['article_id'], row['product_code'], row['prod_name'], row['product_type_name'], row['product_group_name'],
        row['graphical_appearance_name'], row['colour_group_name'], row['perceived_colour_value_name'], 
        row['department_name'], row['index_group_name'], row['section_name'], row['detail_desc'], embedding.tolist()
    )
    
    try:
        cursor.execute(query, values)
        conn.commit()
    except Exception as e:
        print(f"Error inserting data: {e}")
        conn.rollback()
    finally:
        cursor.close()




In [48]:
def main():
    # Add numpy adapter
    add_numpy_adapter()
    
    # Specify the columns to extract
    file_path = '/Users/janezhang/final--project/pgvector/NLP/text-search/text-embedding-filled.csv'
    columns = ['article_id', 'product_code', 'prod_name', 'product_type_name', 'product_group_name', 
               'graphical_appearance_name', 'colour_group_name', 'perceived_colour_value_name', 
               'department_name', 'index_group_name', 'section_name', 'detail_desc']  
    
    # Load the CLIP model
    model = CLIPModel.from_pretrained("openai/clip-vit-base-patch32")
    processor = CLIPProcessor.from_pretrained("openai/clip-vit-base-patch32")
    
    # Connect to the PostgreSQL database
    conn = psycopg2.connect(
        dbname="imageSearch", 
        user="root", 
        password="root", 
        host="localhost", 
        port="5432"
    )
    
    # Read, process, and store data row by row
    for chunk in pd.read_csv(file_path, usecols=columns,dtype={'article_id': str, 'product_code': str}, chunksize=1):
        row = chunk.iloc[0]  # Retrieve a single row of data
        combined_text = ' | '.join(row.values.astype(str))  # Merge all columns of the row into a single string
        
        # Generate the embedding for the row's text
        embedding = generate_clip_embedding(model, processor, combined_text)
        
        # Store the entire row data along with the embedding into pgvector
        store_embedding_in_pgvector(conn, 'text_vector', row, embedding)
    
    # Close the database connection
    conn.close()

if __name__ == '__main__':
    main()


/opt/anaconda3/envs/image-search/lib/python3.12/site-packages/transformers/tokenization_utils_base.py:1617: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be deprecated in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(
